### MF731 HW2 Part1
#### Edited By Xuyang Liu

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
import math

#### 1. VaR for a Portfolio of Microsoft, Apple and Google Stocks.

In [2]:
#(a)
lamda = 0.97
theta = 0.97
data = pd.read_excel('StockData.xlsx',index_col=0)
logret = np.log(data/data.shift(1)).dropna()

In [3]:
# for mu
mu = np.zeros([len(logret),3])
for i in range(1,len(logret)):
    mu[i,:] = lamda * mu[i-1,:] + (1-lamda) * logret.iloc[i,:]
    

In [4]:
mu[-1] # mu for 8/31/21

array([0.00263687, 0.00238145, 0.00312769])

In [5]:
# for the newest covariance matrix
sigma = np.mat([[0,0,0],[0,0,0],[0,0,0]])
for i in range(1,len(logret)+1):
    sigma = theta * sigma + (1-theta) * ( np.mat(logret.iloc[i-1,:].values)-np.mat(mu[i-1,:]) ).T * ( np.mat(logret.iloc[i-1,:].values)-np.mat(mu[i-1,:]) )

In [6]:
sigma # sigma matrix for 8/31/21

matrix([[9.47092473e-05, 8.21428718e-05, 5.37927754e-05],
        [8.21428718e-05, 1.58101963e-04, 7.23027640e-05],
        [5.37927754e-05, 7.23027640e-05, 1.02648243e-04]])

 .

In [7]:
#(b)
alpha = 0.95
m_port = 2.269/(2.269+2.510+1.940)
a_port = 2.510/(2.269+2.510+1.940)
g_port = 1.940/(2.269+2.510+1.940)

theta_ar = np.array([m_port,a_port,g_port])
theta_mat = np.mat([m_port,a_port,g_port])

In [8]:
# For empirical distribution
emp_port = pd.DataFrame()
emp_port["Full"] = -np.dot((np.exp(logret)-1),theta_ar)
emp_port["Lin"] = -np.dot(logret,theta_ar)
emp_port["Sec"] = emp_port["Lin"] - 0.5*np.dot((logret**2),theta_ar)

In [9]:
print("The VaR of emprical full loss is",emp_port.sort_values("Full")["Full"].quantile(0.95)* 1000000  )
print("The VaR of emprical linear loss is",emp_port.sort_values("Lin")["Lin"].quantile(0.95)* 1000000 )
print("The VaR of emprical quadric loss is",emp_port.sort_values("Sec")["Sec"].quantile(0.95)* 1000000 )

The VaR of emprical full loss is 27862.93997656743
The VaR of emprical linear loss is 28262.53177239395
The VaR of emprical quadric loss is 27859.09556465448


In [10]:
# For simulation
# for EWMA X
def loss_VaR(mu,sigma,losstype):
    loss = []
    for i in range(10000):
        X = np.random.multivariate_normal(mu,sigma)
        if losstype == 'Full':
            Full_loss =  - np.dot((np.exp(X)-1),theta_ar) * 1000000
            loss.append(Full_loss)
        elif losstype == 'Linear':
            linear_loss = -np.dot(X , theta_ar) * 1000000
            loss.append(linear_loss)
        elif losstype == 'Second':
            second_loss = -np.dot(X + 0.5 * X**2 , theta_ar) * 1000000
            loss.append(second_loss)
    
    VaR = -(-pd.Series(loss).quantile(alpha) )
    if losstype == 'Linear':
        VaR = (((theta_mat * sigma * theta_mat.T)[0,0]**(1/2) * norm.ppf(alpha)) - np.dot(mu,theta_ar) ) * 1000000
    return loss,VaR

In [11]:
sm_loss_Full, sm_VaR_Full = loss_VaR(mu[-1],sigma,'Full')

In [12]:
sm_loss_linear, sm_VaR_linear = loss_VaR(mu[-1],sigma,'Linear')

In [13]:
sm_loss_second, sm_VaR_second = loss_VaR(mu[-1],sigma,'Second')

In [14]:
print("The VaR of simulation full loss is",sm_VaR_Full)
print("The VaR of simulation linear loss is",sm_VaR_linear)
print("The VaR of simulation quadric loss is",sm_VaR_second)

The VaR of simulation full loss is 12824.363958321215
The VaR of simulation linear loss is 12767.46428767112
The VaR of simulation quadric loss is 12142.339340669829


.

In [15]:
# For standard estimate
stand_mu = logret.mean().values
stand_sigma = logret.cov().values

In [18]:
sm_loss_Full, sm_VaR_Full = loss_VaR(stand_mu,stand_sigma,'Full')
sm_loss_linear, sm_VaR_linear = loss_VaR(stand_mu,stand_sigma,'Linear')
sm_loss_second, sm_VaR_second = loss_VaR(stand_mu,stand_sigma,'Second')
print("The VaR of simulation full loss is",sm_VaR_Full)
print("The VaR of simulation linear loss is",sm_VaR_linear)
print("The VaR of simulation quadric loss is",sm_VaR_second)

The VaR of simulation full loss is 30804.142089729598
The VaR of simulation linear loss is 30953.022771690412
The VaR of simulation quadric loss is 30368.61355517262


From the result above we can see that, the empirical VaR is around 27000, normal VaR using EWMA is about 13000, and normal VaR using standard estimator is about 30000. The reason why EWMA is less

#### 2.VaR and Time Aggregation

In [23]:
#(1)
alpha = 0.95
t = 0
K = 10
delta = 1/252
T = 0.292
S0 = 152.51
k = 170
strike = 170
r = 0.00119
sigma = 0.4907
u = 16.91/100
N = 4000

def BS(t,St,K):
    # BS model, input t, St and K, return put price and Greek delta
    d1 = 1/(sigma * ((T-t)**0.5)) * (np.log(St/K) + (r + 0.5 * sigma**2) * (T-t))
    d2 = d1 - sigma * (T-t)**0.5
    p = St * (norm.cdf(d1)-1) + K * np.exp(-r * (T-t)) * (1-norm.cdf(d2))
    delta_greek = norm.cdf(d1) - 1
    #gamma = norm.pdf(d1)/(St * sigma * (T-t)**0.5)
    #theta = -sigma/(2 * (T-t)**0.5) * St * norm.pdf(d1) + K * r * np.exp(-r * (T-t)) * (1-norm.cdf(d2))
    return p,delta_greek

P0, h0 = BS(t,St,k)

In [24]:
def Loss1day(N):
    # this funciton is not used
    losslst = []
    for i in range(N):
        
        V0 = h0 * S0 + 0 - P0
        S1 = S0 * np.exp((u-1/2 * sigma**2)*delta + sigma*(delta)**0.5 * np.random.normal())
        P1,h1 = BS(t+delta,S1,strike)
        V1 = h0 * S1 +0 -P1
        Loss = V0-V1
        losslst.append(Loss)
        
    return losslst

In [25]:
def Loss10day(N,days):
    losslst = []
    for i in range(N):
        daylosslst = []
        Sk0 = S0
        Pk0,hk0 = BS(t,Sk0,strike)
        Yk0 = 0
        for j in range(days):
            Vk0 = hk0 * Sk0 + Yk0 - Pk0
            Skt = Sk0 * np.exp((u-1/2 * sigma**2)*delta + sigma*(delta)**0.5 * np.random.normal())
            Pkt,hkt = BS(t+(j+1)*delta,Skt,strike)
            Vkt = hk0 * Skt + Yk0 * np.exp(delta * r) - Pkt
            Loss = -(Vkt - Vk0)

            Ykt = Yk0 - (hkt - hk0) * Sk0
            Yk0 = Ykt
            Vk0 = Vkt
            hk0 = hkt
            Sk0 = Skt
            Pk0 = Pkt
            daylosslst.append(Loss)
        dayloss = sum(daylosslst)
        losslst.append(dayloss)
    return losslst          
        

In [26]:
onedayloss = Loss10day(40000,1)
onedayVaR = pd.Series(onedayloss).quantile(0.95) * 100
tendayloss = Loss10day(40000,10)
tendayVaR = pd.Series(tendayloss).quantile(0.95) * 100
direct = onedayVaR * 10**0.5

In [36]:
print("the 1 day VaR is about: ",onedayVaR)
print("the 10 day VaR is about: ", tendayVaR)
print("the result of one day VaR times Sqrt(10): ", direct)

the 1 day VaR is about:  30.529933682155015
the 10 day VaR is about:  89.20855753972728
the result of one day VaR times Sqrt(10):  96.54412724950096


#### 3.  Backtesting VaR.

In [28]:
SP = pd.read_csv('SP_Prices.csv',index_col=0,header=None)
SPret = np.log(SP/SP.shift(1)).dropna()

In [29]:
alpha = 0.95
beta = 0.05
theta = 0.97
lamda = 0.97
M = 1010
N = len(SPret)

# For empirical distribution
Loss = 1 - np.exp(SPret.iloc[:,0])

VaR_emp = []
exc_emp = 0
for i in range(N-M):
    VaR = np.quantile(Loss[i:i+M] , alpha)
    VaR_emp.append(VaR)
    if Loss.iloc[i+M]>VaR_emp[-1]:
        exc_emp += 1

In [30]:
exc_emp

44

In [31]:
# For EWMA updating
mu0 = np.mean(SPret.iloc[0:M-1,0])
sig0 = np.var(SPret.iloc[0:M-1,0])**0.5
VaR_EWMA = []
exc_EWMA = 0

for i in range(1,N-M):
    mu0 = lamda * mu0 + (1-lamda) * SPret.iloc[i+M-1,0]
    sig0 = (theta * sig0**2 + (1-theta) * (SPret.iloc[i+M-1,0]-mu0)**2) **0.5
    VaR = -(np.exp(mu0+sig0 *norm.ppf(1-alpha))-1)
    VaR_EWMA.append(VaR)
    if Loss.iloc[i+M] > VaR_EWMA[-1]:
        exc_EWMA += 1

In [32]:
exc_EWMA # in class we got 79, I think that might because we keep the different decimal.

80

In [33]:
diff = N-M
CI_low = diff * (1-alpha) - norm.ppf(1-beta/2) * np.sqrt(diff*alpha*(1-alpha))
CI_high = diff * (1-alpha) + norm.ppf(1-beta/2) * np.sqrt(diff*alpha*(1-alpha))

In [34]:
diff * (1-alpha) # For Average

75.40000000000006

In [35]:
CI_low,CI_high # For CI

(58.81194118269656, 91.98805881730357)